# 1. IMPORT LIBRARIES

In [1]:
import numpy as np
import pandas as pd
import sys
print('Python version: {}'.format(sys.version))
print('Numpy: {}'.format(np.version))
print('Pandas: {}'.format(pd.__version__))

Python version: 3.10.4 | packaged by conda-forge | (main, Mar 30 2022, 08:38:02) [MSC v.1916 64 bit (AMD64)]
Numpy: <module 'numpy.version' from 'c:\\Users\\PandiyaDS\\anaconda3\\envs\\env1\\lib\\site-packages\\numpy\\version.py'>
Pandas: 1.4.2


In [2]:
# Feature Engineering
from sklearn.feature_selection import SelectKBest, chi2
from scipy import stats as stats

In [3]:
# ML packages
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.tree import DecisionTreeRegressor # Decision Tree
from sklearn.ensemble import RandomForestRegressor # Random Forest
from sklearn import linear_model, preprocessing # Linear Regression
from sklearn.metrics import r2_score
from xgboost import XGBRegressor, plot_importance

In [4]:
# Visual
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as plx
%matplotlib inline

In [5]:
# File system
import os
from os import listdir
from os.path import isfile, join
# Warning
import warnings
warnings.filterwarnings('ignore')

# 2. READ DATA

In [6]:
df = pd.read_csv('train.csv')
pd.set_option('display.max_columns', None)
df.head()
# Attribute Information:
#     1. CRIM      per capita crime rate by town
#     2. ZN        proportion of residential land zoned for lots over 25,000 sq.ft.
#     3. INDUS     proportion of non-retail business acres per town
#     4. CHAS      Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
#     5. NOX       nitric oxides concentration (parts per 10 million)
#     6. RM        average number of rooms per dwelling
#     7. AGE       proportion of owner-occupied units built prior to 1940
#     8. DIS       weighted distances to five Boston employment centres
#     9. RAD       index of accessibility to radial highways
#     10. TAX      full-value property-tax rate per $10,000
#     11. PTRATIO  pupil-teacher ratio by town
#     12. B        1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
#     13. LSTAT    % lower status of the population
#     14. MEDV     Median value of owner-occupied homes in $1000's

,ID,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0,0.95577,0.0,8.14,0,0.538,6.047,88.8,4.4534,4,307.0,21.0,306.38,17.28,14.8
1,1,0.02875,28.0,15.04,0,0.464,6.211,28.9,3.6659,4,270.0,18.2,396.33,6.21,25.0
2,2,1.22358,0.0,19.58,0,0.605,6.943,97.4,1.8773,5,403.0,14.7,363.43,4.59,41.3
3,3,5.66637,0.0,18.10,0,0.740,6.219,100.0,2.0048,24,666.0,20.2,395.69,16.59,18.4
4,4,0.04544,0.0,3.24,0,0.460,6.144,32.2,5.8736,4,430.0,16.9,368.57,9.09,19.8


In [11]:
df.shape

(400, 15)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 15 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   ID       400 non-null    int64  
 1   CRIM     400 non-null    float64
 2   ZN       400 non-null    float64
 3   INDUS    400 non-null    float64
 4   CHAS     400 non-null    int64  
 5   NOX      400 non-null    float64
 6   RM       400 non-null    float64
 7   AGE      400 non-null    float64
 8   DIS      400 non-null    float64
 9   RAD      400 non-null    int64  
 10  TAX      400 non-null    float64
 11  PTRATIO  400 non-null    float64
 12  B        400 non-null    float64
 13  LSTAT    400 non-null    float64
 14  MEDV     400 non-null    float64
dtypes: float64(12), int64(3)
memory usage: 47.0 KB


In [12]:
df.isnull().sum()

ID         0
CRIM       0
ZN         0
INDUS      0
CHAS       0
NOX        0
RM         0
AGE        0
DIS        0
RAD        0
TAX        0
PTRATIO    0
B          0
LSTAT      0
MEDV       0
dtype: int64

In [14]:
df.describe().round(2) # Checking the data distribution

,ID,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
count,400.00,400.00,400.00,400.00,400.00,400.00,400.00,400.00,400.00,400.00,400.00,400.00,400.00,400.00,400.00
mean,199.50,3.76,10.97,10.94,0.07,0.55,6.29,68.09,3.82,9.46,403.80,18.46,359.46,12.67,22.48
std,115.61,9.16,22.80,6.85,0.26,0.12,0.71,28.39,2.13,8.69,169.66,2.15,86.73,7.21,9.22
min,0.00,0.01,0.00,0.46,0.00,0.38,4.14,2.90,1.13,1.00,187.00,12.60,0.32,1.92,5.00
25%,99.75,0.08,0.00,5.13,0.00,0.45,5.88,42.38,2.11,4.00,277.00,17.40,376.12,6.99,17.10
50%,199.50,0.24,0.00,8.56,0.00,0.53,6.21,76.95,3.27,5.00,329.00,18.95,391.58,10.88,21.00
75%,299.25,3.54,12.50,18.10,0.00,0.62,6.62,93.82,5.21,24.00,666.00,20.20,396.28,16.91,25.00
max,399.00,88.98,95.00,27.74,1.00,0.87,8.78,100.00,12.13,24.00,711.00,22.00,396.90,37.97,50.00


In [17]:
# Checking correlation between columns in (%)
df.corr().round(2)*100

,ID,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
ID,100.0,-1.0,-4.0,1.0,-3.0,-3.0,6.0,-3.0,1.0,-3.0,-1.0,6.0,-3.0,-2.0,1.0
CRIM,-1.0,100.0,-19.0,41.0,-6.0,42.0,-21.0,35.0,-38.0,62.0,58.0,28.0,-36.0,44.0,-39.0
ZN,-4.0,-19.0,100.0,-52.0,-4.0,-50.0,29.0,-55.0,66.0,-30.0,-30.0,-37.0,16.0,-40.0,34.0
INDUS,1.0,41.0,-52.0,100.0,8.0,76.0,-40.0,63.0,-70.0,61.0,73.0,37.0,-34.0,62.0,-50.0
CHAS,-3.0,-6.0,-4.0,8.0,100.0,8.0,9.0,8.0,-10.0,-2.0,-5.0,-14.0,4.0,-6.0,18.0
NOX,-3.0,42.0,-50.0,76.0,8.0,100.0,-28.0,74.0,-77.0,60.0,66.0,15.0,-38.0,62.0,-43.0
RM,6.0,-21.0,29.0,-40.0,9.0,-28.0,100.0,-20.0,17.0,-19.0,-28.0,-34.0,12.0,-63.0,72.0
AGE,-3.0,35.0,-55.0,63.0,8.0,74.0,-20.0,100.0,-75.0,46.0,50.0,23.0,-27.0,59.0,-35.0
DIS,1.0,-38.0,66.0,-70.0,-10.0,-77.0,17.0,-75.0,100.0,-49.0,-52.0,-19.0,27.0,-50.0,24.0
RAD,-3.0,62.0,-30.0,61.0,-2.0,60.0,-19.0,46.0,-49.0,100.0,91.0,46.0,-40.0,50.0,-39.0


## Observation
----------------
1. Rows: 400 and Columns: 15
2. Non of the column have Null values.
3. All columns have data type float or int type.
4. Median value of the house range between 5K $ to 50k $.
5. Except column ID, MEDV have strong correlation with other independent variables.